In [1]:
import numpy as np

with open("movie-xids.npy", "rb") as f:
    Xids = np.load(f, allow_pickle=True)
with open("movie-xmask.npy", "rb") as f:
    Xmask = np.load(f, allow_pickle=True)
with open("movie-labels.npy", "rb") as f:
    labels = np.load(f, allow_pickle=True)

In [2]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))
dataset.take(1)

<TakeDataset shapes: ((160,), (160,), (6,)), types: (tf.int32, tf.int32, tf.float64)>

In [3]:
def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {"input_ids": input_ids,
            "attention_mask": masks}, labels

In [4]:
dataset = dataset.map(map_func)
dataset.take(1)

<TakeDataset shapes: ({input_ids: (160,), attention_mask: (160,)}, (6,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [5]:
batch_size=32
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

dataset.take(1)

<TakeDataset shapes: ({input_ids: (32, 160), attention_mask: (32, 160)}, (32, 6)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [8]:
Xids[0], Xids[0].shape

(array([     2,  13158,  62480,  78805,   3671,  16421,   2767,   3100,
          5855, 106576, 120739,  67719,     47,   7647,   3156,   2416,
          1019,      3,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0, 

In [9]:
split = 0.9
size = int((Xids.shape[0] / batch_size) * split)
size

2340

In [10]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

len(dataset), len(train_ds), len(val_ds)

(2600, 2340, 260)

In [11]:
tf.data.experimental.save(train_ds, "train")
tf.data.experimental.save(val_ds, "val")

In [12]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(32, 160), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(32, 160), dtype=tf.int32, name=None)},
 TensorSpec(shape=(32, 6), dtype=tf.float64, name=None))

In [13]:
val_ds.element_spec

({'input_ids': TensorSpec(shape=(32, 160), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(32, 160), dtype=tf.int32, name=None)},
 TensorSpec(shape=(32, 6), dtype=tf.float64, name=None))

In [14]:
train_ds.element_spec == val_ds.element_spec

True